### ResNet18 모델 정의 및 인스턴스 초기화

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우 (입력값 =/= 출력값 dim)
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False) #conv layer 1개
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1) #basic block 당 conv layer 2개
                                                                            #그게 2개이므로 총 4개
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        #총 16개의 conv layer
        
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)#첫번째 conv 연산에 의해서만 너비와 높이가 줄어들 수 있도록
                                                   #나머지는 stride = 1 로하여 같은 너비와 높이 유지
                                                #filter 개수가 증가할때마다 너비와 높이는 줄어들수 있게 설정
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

### 데이터셋(Dataset) 다운로드 및 불러오기

In [2]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
  
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


### 환경 설정 및 학습(Training) 함수 정의


In [3]:
device = 'cuda:1'

net = ResNet18()
net = net.to(device)
# net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')

#learning rate를 바꾸기
def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10 #1/10 로 줄이기
    if epoch >= 150:
        lr /= 10  #1/10 로 줄이기
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

### 학습(Training) 진행

In [4]:
for epoch in range(0, 150):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.0859375
Current benign train loss: 2.372758626937866

Current batch: 100
Current benign train accuracy: 0.296875
Current benign train loss: 1.9468672275543213

Current batch: 200
Current benign train accuracy: 0.3203125
Current benign train loss: 1.826780915260315

Current batch: 300
Current benign train accuracy: 0.4453125
Current benign train loss: 1.5565731525421143

Total benign train accuarcy: 32.01
Total benign train loss: 752.5413776636124

[ Test epoch: 0 ]

Test accuarcy: 34.28
Test average loss: 0.018626053309440612
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.3515625
Current benign train loss: 1.6823360919952393

Current batch: 100
Current benign train accuracy: 0.4140625
Current benign train loss: 1.5369478464126587

Current batch: 200
Current benign train accuracy: 0.3828125
Current benign train loss: 1.6747404336929321

Current batch: 300
Current benign train accu


Current batch: 200
Current benign train accuracy: 0.9140625
Current benign train loss: 0.3190076947212219

Current batch: 300
Current benign train accuracy: 0.859375
Current benign train loss: 0.4360516369342804

Total benign train accuarcy: 87.422
Total benign train loss: 141.26227900385857

[ Test epoch: 13 ]

Test accuarcy: 84.04
Test average loss: 0.004586342026293278
Model Saved!

[ Train epoch: 14 ]

Current batch: 0
Current benign train accuracy: 0.859375
Current benign train loss: 0.33361300826072693

Current batch: 100
Current benign train accuracy: 0.890625
Current benign train loss: 0.3446691334247589

Current batch: 200
Current benign train accuracy: 0.8671875
Current benign train loss: 0.3565326929092407

Current batch: 300
Current benign train accuracy: 0.875
Current benign train loss: 0.33458268642425537

Total benign train accuarcy: 88.508
Total benign train loss: 130.4381532073021

[ Test epoch: 14 ]

Test accuarcy: 85.15
Test average loss: 0.004593064898252487
Model 


Total benign train accuarcy: 93.09
Total benign train loss: 76.47285750880837

[ Test epoch: 26 ]

Test accuarcy: 87.86
Test average loss: 0.004017341405898333
Model Saved!

[ Train epoch: 27 ]

Current batch: 0
Current benign train accuracy: 0.953125
Current benign train loss: 0.16903628408908844

Current batch: 100
Current benign train accuracy: 0.9609375
Current benign train loss: 0.14120014011859894

Current batch: 200
Current benign train accuracy: 0.921875
Current benign train loss: 0.23545478284358978

Current batch: 300
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2854923605918884

Total benign train accuarcy: 93.434
Total benign train loss: 72.86000822484493

[ Test epoch: 27 ]

Test accuarcy: 87.32
Test average loss: 0.004171158963441848
Model Saved!

[ Train epoch: 28 ]

Current batch: 0
Current benign train accuracy: 0.9375
Current benign train loss: 0.16964229941368103

Current batch: 100
Current benign train accuracy: 0.9453125
Current benign tra


Current batch: 0
Current benign train accuracy: 0.953125
Current benign train loss: 0.16463196277618408

Current batch: 100
Current benign train accuracy: 0.9375
Current benign train loss: 0.11939972639083862

Current batch: 200
Current benign train accuracy: 0.9765625
Current benign train loss: 0.07268824428319931

Current batch: 300
Current benign train accuracy: 0.9609375
Current benign train loss: 0.1375098079442978

Total benign train accuarcy: 95.044
Total benign train loss: 55.30249237269163

[ Test epoch: 40 ]

Test accuarcy: 89.53
Test average loss: 0.0035521504499018194
Model Saved!

[ Train epoch: 41 ]

Current batch: 0
Current benign train accuracy: 0.921875
Current benign train loss: 0.2517731785774231

Current batch: 100
Current benign train accuracy: 0.9375
Current benign train loss: 0.1668943464756012

Current batch: 200
Current benign train accuracy: 0.921875
Current benign train loss: 0.24546948075294495

Current batch: 300
Current benign train accuracy: 0.8984375
Cu


Current batch: 200
Current benign train accuracy: 0.984375
Current benign train loss: 0.0779825747013092

Current batch: 300
Current benign train accuracy: 0.96875
Current benign train loss: 0.09235793352127075

Total benign train accuarcy: 95.864
Total benign train loss: 45.35042480751872

[ Test epoch: 53 ]

Test accuarcy: 89.9
Test average loss: 0.0034796714305877687
Model Saved!

[ Train epoch: 54 ]

Current batch: 0
Current benign train accuracy: 0.9609375
Current benign train loss: 0.11806207150220871

Current batch: 100
Current benign train accuracy: 0.96875
Current benign train loss: 0.09970135241746902

Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train loss: 0.05794654041528702

Current batch: 300
Current benign train accuracy: 0.9453125
Current benign train loss: 0.15716679394245148

Total benign train accuarcy: 95.838
Total benign train loss: 46.84988447278738

[ Test epoch: 54 ]

Test accuarcy: 88.9
Test average loss: 0.00379784282296896
Mode

...


Current batch: 0
Current benign train accuracy: 0.9453125
Current benign train loss: 0.16918855905532837

Current batch: 100
Current benign train accuracy: 0.9765625
Current benign train loss: 0.061598990112543106

Current batch: 200
Current benign train accuracy: 0.96875
Current benign train loss: 0.1062842309474945

Current batch: 300
Current benign train accuracy: 0.9609375
Current benign train loss: 0.14277620613574982

Total benign train accuarcy: 96.822
Total benign train loss: 35.848410706967115

[ Test epoch: 93 ]

Test accuarcy: 90.31
Test average loss: 0.0035516177073121073
Model Saved!

[ Train epoch: 94 ]

Current batch: 0
Current benign train accuracy: 0.9765625
Current benign train loss: 0.05992770940065384

Current batch: 100
Current benign train accuracy: 0.9609375
Current benign train loss: 0.09161782264709473

Current batch: 200
Current benign train accuracy: 0.96875
Current benign train loss: 0.06697303056716919

Current batch: 300
Current benign train accuracy: 0.9


Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.004368633031845093

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.004085324704647064

Total benign train accuarcy: 99.802
Total benign train loss: 3.2143724812194705

[ Test epoch: 106 ]

Test accuarcy: 94.15
Test average loss: 0.002269815032184124
Model Saved!

[ Train epoch: 107 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.004713580012321472

Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.009892608970403671

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.003967486321926117

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.007121510803699493

Total benign train accuarcy: 99.858
Total benign train loss: 2.7282355692004785

[ Test epoch: 107 ]

Test accuarcy: 94.25
Test average loss: 0.0022928857080638408
Model Saved!




Test accuarcy: 94.28
Test average loss: 0.002318046719953418
Model Saved!

[ Train epoch: 120 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0008047595620155334

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0008320808410644531

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.005268074572086334

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0008035264909267426

Total benign train accuarcy: 99.962
Total benign train loss: 1.1537269052059855

[ Test epoch: 120 ]

Test accuarcy: 94.39
Test average loss: 0.0023228620365262033
Model Saved!

[ Train epoch: 121 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0032198503613471985

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0028932392597198486

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.


Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.00036306679248809814

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0004734247922897339

Total benign train accuarcy: 99.978
Total benign train loss: 0.740147965028882

[ Test epoch: 133 ]

Test accuarcy: 94.36
Test average loss: 0.002331472597271204
Model Saved!

[ Train epoch: 134 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0010045245289802551

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0015066489577293396

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.000988110899925232

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.003355119377374649

Total benign train accuarcy: 99.986
Total benign train loss: 0.6549968063773122

[ Test epoch: 134 ]

Test accuarcy: 94.57
Test average loss: 0.0023108897380530835
Model Saved!

[ 


Test accuarcy: 94.61
Test average loss: 0.0022983204532414673
Model Saved!

[ Train epoch: 147 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0012747310101985931

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0014387443661689758

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0003022439777851105

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0013523101806640625

Total benign train accuarcy: 99.988
Total benign train loss: 0.546846938872477

[ Test epoch: 147 ]

Test accuarcy: 94.65
Test average loss: 0.0022674564827233554
Model Saved!

[ Train epoch: 148 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.000661998987197876

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0018216148018836975

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.